## Demand and Supply analysis for uber

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
pio.templates.default = "plotly_white"
data = pd.read_csv('Uber Request Data.csv')
print(data.head())

   Request id Pickup point  Driver id          Status    Request timestamp  \
0         619      Airport        1.0  Trip Completed      11/7/2016 11:51   
1         867      Airport        1.0  Trip Completed      11/7/2016 17:57   
2        1807         City        1.0  Trip Completed       12/7/2016 9:17   
3        2532      Airport        1.0  Trip Completed      12/7/2016 21:08   
4        3112         City        1.0  Trip Completed  13-07-2016 08:33:16   

        Drop timestamp  
0      11/7/2016 13:00  
1      11/7/2016 18:47  
2       12/7/2016 9:58  
3      12/7/2016 22:03  
4  13-07-2016 09:25:47  


## Cheking null values in our data set

In [23]:
print(data.isnull().sum())

Request id              0
Pickup point            0
Driver id            2650
Status                  0
Request timestamp       0
Drop timestamp       3914
dtype: int64


## cheking number of our records

In [24]:
rows=data.shape[0]
columns=data.shape[1]
print(rows)
print(columns)



6745
6


## Cheking basic information about our data set

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Request id         6745 non-null   int64  
 1   Pickup point       6745 non-null   object 
 2   Driver id          4095 non-null   float64
 3   Status             6745 non-null   object 
 4   Request timestamp  6745 non-null   object 
 5   Drop timestamp     2831 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 316.3+ KB


## How many trips were completed and cancelled

In [26]:
data.Status.value_counts()

Trip Completed       2831
No Cars Available    2650
Cancelled            1264
Name: Status, dtype: int64

## Rename columns for easy manipulation

In [27]:
## first we rename columns in the data set
newData=data.rename(columns={'Request id': 'Request_id', 'Driver id': 'Driver_id','Pickup point':'Pickup_point',
                             'Request timestamp':'Request_timestamp','Drop timestamp':'Drop_timestamp'})
newData.head()

,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47


## Number of rides  by each driver  in our data set

In [28]:
rideNum=newData.groupby('Driver_id')['Request_id'].nunique() 
print(rideNum)

Driver_id
1.0      13
2.0      13
3.0      14
4.0      15
5.0      13
         ..
296.0    14
297.0    15
298.0    15
299.0    12
300.0     6
Name: Request_id, Length: 300, dtype: int64


## The driver with the highest number of rides


In [29]:
rideNum=newData.groupby('Driver_id')['Request_id'].nunique() 
print(rideNum.max())

22


## The driver with the lowest number of rides

In [30]:
rideNum=newData.groupby('Driver_id')['Request_id'].nunique() 
print(rideNum.min())

6


## List of Driver with completed, cancelled and unavaliable

In [31]:
StatusSummary = newData.pivot_table(values='Status', index=['Driver_id', 'Request_id'],
                    columns=['Pickup_point'], aggfunc=np.sum)
print(StatusSummary)

Pickup_point                 Airport            City
Driver_id Request_id                                
1.0       619         Trip Completed             NaN
          867         Trip Completed             NaN
          1807                   NaN  Trip Completed
          2532        Trip Completed             NaN
          2905                   NaN       Cancelled
...                              ...             ...
300.0     2924                   NaN  Trip Completed
          3324        Trip Completed             NaN
          3935        Trip Completed             NaN
          4225                   NaN       Cancelled
          5844        Trip Completed             NaN

[4095 rows x 2 columns]


## Where did most drop off occur

In [33]:
newData.Pickup_point.value_counts()

City       3507
Airport    3238
Name: Pickup_point, dtype: int64

## Changing pickup and drop time to date format for easy analysis

In [37]:
newData['Request_timestamp'] = pd.to_datetime(newData['Request_timestamp'],
                                       errors='coerce')
newData['Drop_timestamp'] = pd.to_datetime(newData['Drop_timestamp'],
                                     errors='coerce')
newData.head()

,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47


## Splitting the Reuest time to date and time column and then converting the time into four different categories i.e. Morning, Afternoon, Evening, Night

In [40]:
from datetime import datetime
 
newData['date'] = pd.DatetimeIndex(newData['Request_timestamp']).date
newData['time'] = pd.DatetimeIndex(newData['Request_timestamp']).hour
 
#changing into categories of day and night
newData['day-night'] = pd.cut(x=newData['time'],
                              bins = [0,10,15,19,24],
                              labels = ['Morning','Afternoon','Evening','Night'])
newData.head()


,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp,date,time,day-night
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,2016-11-07,11,Afternoon
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,2016-11-07,17,Evening
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,2016-12-07,9,Morning
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,2016-12-07,21,Night
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,2016-07-13,8,Morning


## Droping columns with null values

In [42]:
## data without columns with null values
newData.dropna(inplace=True)

## Removing duplicates from rows

In [43]:
newData.drop_duplicates(inplace=True)

## Time taken by each ride


In [47]:
newData['Time-diff']=
newData.head()

,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp,date,time,day-night,Time-diff
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,2016-11-07,11,Afternoon,0 days 01:09:00
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,2016-11-07,17,Evening,0 days 00:50:00
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,2016-12-07,9,Morning,0 days 00:41:00
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,2016-12-07,21,Night,0 days 00:55:00
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,2016-07-13,8,Morning,0 days 00:52:31


## Which weekdays had the highest average fares?